In [1]:
import numpy as np
import nibabel as nib
import torch
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

print('imported')

imported


### Loading data

In [2]:
ROOT_DIR = '../common_data/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021/'

n_samples = 1000
flair, mask, t1, t2, t1ce = list(), list(), list(), list(), list()
for instance in tqdm(os.listdir(ROOT_DIR)[: n_samples], position=0, leave=True):
    mask_model = nib.load(os.path.join(ROOT_DIR, instance, instance + '_seg.nii.gz')).get_fdata()
    # selecting best slice
    window = 0
    best_slice_idx = np.argmax(np.sum(mask_model, axis=(0, 1)))

    # loading whole 3D model
    flair_model = nib.load(os.path.join(ROOT_DIR, instance, instance + '_flair.nii.gz')).dataobj[:, :, best_slice_idx - window:best_slice_idx + window + 1]
    t1_model = nib.load(os.path.join(ROOT_DIR, instance, instance + '_t1.nii.gz')).dataobj[:, :, best_slice_idx - window:best_slice_idx + window + 1]
    t2_model = nib.load(os.path.join(ROOT_DIR, instance, instance + '_t2.nii.gz')).dataobj[:, :, best_slice_idx - window:best_slice_idx + window + 1]
    t1ce_model = nib.load(os.path.join(ROOT_DIR, instance, instance + '_t1ce.nii.gz')).dataobj[:, :, best_slice_idx - window:best_slice_idx + window + 1]
    mask_model = mask_model[:, :, best_slice_idx - window:best_slice_idx + window + 1]

    # appending 10% best slices
    flair.extend([flair_model[:, :, slice] for slice in range(flair_model.shape[2])])
    t1.extend([t1_model[:, :, slice] for slice in range(t1_model.shape[2])])
    t2.extend([t2_model[:, :, slice] for slice in range(t2_model.shape[2])])
    t1ce.extend([t1ce_model[:, :, slice] for slice in range(t1ce_model.shape[2])])
    mask.extend([mask_model[:, :, slice] for slice in range(mask_model.shape[2])])

flair, t1, t2, t1ce, mask = np.array(flair), np.array(t1), np.array(t2), np.array(t1ce), np.array(mask)

# shuffle
idx = np.arange(flair.shape[0])
np.random.shuffle(idx)
flair, mask, t1, t2, t1ce = flair[idx], mask[idx], t1[idx], t2[idx], t1ce[idx]

print('Flair shape: ', flair.shape)

np.savez_compressed('data/brats_data.npz', flair=flair, mask=mask, t1=t1, t2=t2, t1ce=t1ce)
print('data saved')

100%|██████████| 1000/1000 [03:39<00:00,  4.55it/s]


Flair shape:  (1000, 240, 240)
data saved


In [33]:
np.savez_compressed('data/brats_data.npz', flair=flair, mask=mask, t1=t1, t2=t2, t1ce=t1ce)
print('data saved')

data saved
